In [10]:
import os

In [11]:
%pwd

'e:\\'

In [14]:
os.chdir("E:/Workspace/End-to-End-Mlops")

In [15]:
%pwd

'E:\\Workspace\\End-to-End-Mlops'

In [16]:
os.getcwd()

'E:\\Workspace\\End-to-End-Mlops'

In [67]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataInjectionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path


In [68]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml,create_directories,load_json

In [69]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH,schema_file_path=SCHEMA_FILE_PATH,para_file_path=PARAMS_FILE_PATH):

        
        self.config = read_yaml(config_file_path)
        self.schema = read_yaml(schema_file_path)
        self.params = read_yaml(para_file_path)

        create_directories([self.config.artifacts_root])


    def get_data_injection_config(self)->DataInjectionConfig:
        
        config=self.config.data_injections

        create_directories([config.root_dir])

        data_injection_config= DataInjectionConfig(
        root_dir=config.root_dir,
        source_URL=config.source_URL,
        local_data_file=config.local_data_file,
        unzip_dir=config.unzip_dir
        )

        return data_injection_config


In [70]:
import os
import urllib.request as request
import zipfile
from src.mlProject.logging import logger
from src.mlProject.utils.common import get_size


In [95]:
class DataInjection:
    def __init__(self,config:DataInjectionConfig):

        self.config=config

    def download_file(self):

        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file 
                )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f" File already exists of size:{ get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):

        """
        zip_file_path:str
        Extracts the zip file into the data directory
        function returns None

         """
        
        unzip_path=self.config.unzip_dir
        
        # Create the directory if it doesn't exist

        if not os.path.exists(unzip_path):
            try:
                os.makedirs(unzip_path,exist_ok=True)
                print(f"Directory '{unzip_path}' created successfully.")
            except PermissionError as e:
                print(f"Permission denied: {e}")
        
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
                print("File extracted successfully.")
        except zipfile.BadZipFile:
            logger.info("Error: The file is not a valid zip file or is corrupted.")

        



In [96]:
try:
    config=ConfigurationManager()
    data_injection_config=config.get_data_injection_config()
    data_injection=DataInjection(config=data_injection_config)
    data_injection.download_file()
    data_injection.extract_zip_file()
except Exception as e:
    raise e


--- Logging error ---
Traceback (most recent call last):
  File "e:\ML_Project\ml_project\lib\logging\__init__.py", line 1082, in emit
    stream.write(msg + self.terminator)
AttributeError: 'str' object has no attribute 'write'
Call stack:
  File "e:\ML_Project\ml_project\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "e:\ML_Project\ml_project\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\HP PC\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\HP PC\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\HP PC\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "C:\Users\HP PC\AppData\Roaming\Python\Python39\site-packages\tornado\platform\asyncio.py", lin

In [97]:
# try:
#     with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
#                 zip_ref.extractall('End-to-End-Mlops')
#                 print("File extracted successfully.")
# except zipfile.BadZipFile:
#             logger.info("Error: The file is not a valid zip file or is corrupted.")

File extracted successfully.
